# TeleChurn Predictor: Feature Engineering (Improved)

This notebook demonstrates the enhanced feature engineering process for telecom customer churn prediction. We'll apply the improved feature engineering module to create advanced features with strong predictive power for customer churn.

## 1. Setup and Data Loading

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
import os

# Import our feature engineering module
import sys
sys.path.append('../scripts')
from feature_engineering import FeatureEngineer, get_feature_importances, calculate_all_ivs

# Configure visualizations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette('viridis')

# Ignore warnings
warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Load the preprocessed data
base_dir = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
processed_data_dir = os.path.join(base_dir, "data", "processed")
train_file = "preprocessed_cell2celltrain.csv"
holdout_file = "preprocessed_cell2cellholdout.csv"

train_data = pd.read_csv(os.path.join(processed_data_dir, train_file))
holdout_data = pd.read_csv(os.path.join(processed_data_dir, holdout_file))

print(f"Training data shape: {train_data.shape}")
print(f"Holdout data shape: {holdout_data.shape}")

In [ ]:
# Display the first few rows of the training data
train_data.head()

## 2. Applying Enhanced Feature Engineering

We'll use our improved feature engineering module with the following enhancements:
1. Explicit exclusion of ID columns (CustomerID)
2. Model-based feature selection for better results
3. New feature categories: customer value, trends, risk scores, and interactions

In [ ]:
# Convert Churn to numeric if it's a string
if train_data['Churn'].dtype == 'object':
    print("Converting Churn from string to numeric...")
    # Map 'Yes'/'No' to 1/0
    train_data['Churn'] = train_data['Churn'].map({'Yes': 1, 'No': 0})

In [ ]:
# Initialize the feature engineer with explicit ID columns to exclude
feature_eng = FeatureEngineer(
    remove_correlated=True, 
    correlation_threshold=0.85,
    id_columns=['CustomerID'],  # Explicitly exclude CustomerID from feature engineering
    selection_method='model_based'  # Use model-based feature selection for better results
)

# Apply feature engineering to training data
train_featured = feature_eng.fit_transform(train_data.copy())

# Apply feature engineering to holdout data
holdout_featured = feature_eng.transform(holdout_data.copy())

# Print shape comparison
print(f"Original training data shape: {train_data.shape}")
print(f"Engineered training data shape: {train_featured.shape}")
print(f"\nOriginal holdout data shape: {holdout_data.shape}")
print(f"Engineered holdout data shape: {holdout_featured.shape}")

In [ ]:
# Get list of new features created
original_features = set(train_data.columns)
engineered_features = set(train_featured.columns)
new_features = engineered_features - original_features

print(f"Number of new features created: {len(new_features)}")
print("\nNew features:")
for feature in sorted(new_features):
    print(f"- {feature}")

## 3. Categorizing and Exploring New Features

Let's categorize our features to better understand the different types of engineered features.

In [ ]:
# Function to categorize features
def categorize_features(df):
    behavioral_features = [col for col in df.columns if col in [
        'CustomerCareCallsPerMonth', 'TotalProblemCalls', 'ProblemCallsPerMonth',
        'TotalCalls', 'CallsPerMonth', 'SpecialFeaturesUsed',
        'RetentionCallsPerMonth', 'RetentionSuccessRate'
    ]]
    
    usage_pattern_features = [col for col in df.columns if col in [
        'PeakCallsRatio', 'RevenuePerMinute', 'PremiumServiceUsage',
        'RoamingServiceUsage', 'HasOverages'
    ]]
    
    change_features = [col for col in df.columns if col in [
        'NormalizedChangeMinutes', 'NormalizedChangeRevenues',
        'ConsistentChangeDirection', 'LargeNegativeChange',
        'TenureBucket', 'IsNewCustomer'
    ]]
    
    ratio_features = [col for col in df.columns if col in [
        'ARPM', 'ProblemCallRatio', 'RetentionToServiceRatio',
        'EquipmentLifeRatio', 'RecurringRevenueRatio'
    ]]
    
    profile_features = [col for col in df.columns if col in [
        'TechSavvyScore', 'HasChildren', 'IsMultiPersonHH',
        'AvgHHAge', 'AgeSegment', 'HasMadeReferrals',
        'HighReferrer', 'CreditScore', 'HighIncome', 'LowIncome'
    ]]
    
    # New feature categories
    customer_value_features = [col for col in df.columns if col in [
        'CustomerLifetimeValue', 'RevenueStability', 'RevenuePerSubscription',
        'HighValueCustomer'
    ]]
    
    trend_features = [col for col in df.columns if col in [
        'UsageTrendDirection', 'RevenueTrendDirection', 'UsageAcceleration',
        'RevenueAcceleration', 'TrendConsistency'
    ]]
    
    risk_features = [col for col in df.columns if col in [
        'ServiceQualityRiskScore', 'FinancialRiskScore', 'EngagementRiskScore',
        'ChurnRiskScore'
    ]]
    
    interaction_features = [col for col in df.columns if col in [
        'NewCustomerWithProblems', 'HighValueCustomerWithProblems',
        'ServiceIssuesWithDecliningUsage', 'FailedRetentionAttempts',
        'TechSavvyWithProblems'
    ]]
    
    return {
        'Behavioral Features': behavioral_features,
        'Usage Pattern Features': usage_pattern_features,
        'Change Features': change_features,
        'Ratio Features': ratio_features,
        'Profile Features': profile_features,
        'Customer Value Features': customer_value_features,
        'Trend Features': trend_features,
        'Risk Features': risk_features,
        'Interaction Features': interaction_features
    }

In [ ]:
# Categorize the engineered features
feature_categories = categorize_features(train_featured)

# Print feature counts by category
for category, features in feature_categories.items():
    print(f"{category}: {len(features)} features")
    for feature in features:
        print(f"  - {feature}")

## 4. Feature Selection and Importance

Let's examine which features were selected by our model-based feature selection process.

In [ ]:
# Get selected features
selected_features = feature_eng.selected_features

if selected_features:
    print(f"Number of selected features: {len(selected_features)}")
    print("\nSelected features:")
    for feature in sorted(selected_features):
        print(f"- {feature}")

In [ ]:
# Prepare data for modeling
train_featured_model = train_featured.copy()

# Check for categorical columns
categorical_cols = [col for col in train_featured_model.columns 
                   if train_featured_model[col].dtype == 'object' or 
                   train_featured_model[col].dtype.name == 'category']

# Encode categorical columns
for col in categorical_cols:
    if col != 'Churn':
        le = LabelEncoder()
        train_featured_model[col] = le.fit_transform(train_featured_model[col].astype(str))

# Ensure target is binary numeric
if 'Churn' in train_featured_model.columns and train_featured_model['Churn'].dtype == 'object':
    train_featured_model['Churn'] = train_featured_model['Churn'].map({'Yes': 1, 'No': 0})

# Prepare X and y
X = train_featured_model.drop('Churn', axis=1) if 'Churn' in train_featured_model.columns else train_featured_model
y = train_featured_model['Churn'] if 'Churn' in train_featured_model.columns else None

In [ ]:
if y is not None:
    # Train a Random Forest model with class weights to balance precision and recall
    rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
    rf.fit(X, y)
    
    # Get feature importances
    feature_importance_df = get_feature_importances(rf, X.columns)
    
    # Display top features
    print("Top 20 features by importance:")
    display(feature_importance_df.head(20))
    
    # Plot feature importances
    plt.figure(figsize=(12, 10))
    sns.barplot(x='Importance', y='Feature', data=feature_importance_df.head(20))
    plt.title('Top 20 Feature Importances')
    plt.tight_layout()
    plt.show()

## 5. Predictive Power Demonstration

Let's compare the predictive power of models trained with original features vs. engineered features.

In [ ]:
# Function to evaluate model performance
def evaluate_model(X_train, y_train, X_test, y_test, model_name, class_weight=None):
    # Train model
    model = RandomForestClassifier(n_estimators=100, class_weight=class_weight, random_state=42)
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    auc = roc_auc_score(y_test, y_prob)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"AUC: {auc:.4f}")
    print(f"Accuracy: {report['accuracy']:.4f}")
    print(f"Precision (Class 1): {report['1']['precision']:.4f}")
    print(f"Recall (Class 1): {report['1']['recall']:.4f}")
    print(f"F1-Score (Class 1): {report['1']['f1-score']:.4f}")
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    return {
        'model': model,
        'auc': auc,
        'accuracy': report['accuracy'],
        'precision': report['1']['precision'],
        'recall': report['1']['recall'],
        'f1': report['1']['f1-score']
    }

In [ ]:
# Prepare original data for modeling
train_data_model = train_data.copy()

# Handle categorical columns in original data
categorical_cols_orig = [col for col in train_data_model.columns 
                        if train_data_model[col].dtype == 'object' or 
                        train_data_model[col].dtype.name == 'category']

for col in categorical_cols_orig:
    if col != 'Churn':
        le = LabelEncoder()
        train_data_model[col] = le.fit_transform(train_data_model[col].astype(str))

# Ensure target is binary numeric
if train_data_model['Churn'].dtype == 'object':
    train_data_model['Churn'] = train_data_model['Churn'].map({'Yes': 1, 'No': 0})

# Split data
X_orig = train_data_model.drop(['CustomerID', 'Churn'], axis=1)
y_orig = train_data_model['Churn']
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(
    X_orig, y_orig, test_size=0.3, random_state=42, stratify=y_orig
)

In [ ]:
# Split engineered data
X_eng = train_featured_model.drop(['Churn'], axis=1) if 'Churn' in train_featured_model.columns else train_featured_model
y_eng = train_featured_model['Churn'] if 'Churn' in train_featured_model.columns else y_orig
X_train_eng, X_test_eng, y_train_eng, y_test_eng = train_test_split(
    X_eng, y_eng, test_size=0.3, random_state=42, stratify=y_eng
)

In [ ]:
# Evaluate model with original features
original_results = evaluate_model(
    X_train_orig, y_train_orig, 
    X_test_orig, y_test_orig,
    "Original Features Model"
)

In [ ]:
# Evaluate model with engineered features
engineered_results = evaluate_model(
    X_train_eng, y_train_eng, 
    X_test_eng, y_test_eng,
    "Engineered Features Model",
    class_weight='balanced'  # Use class weights to balance precision and recall
)

In [ ]:
# Compare model performance
metrics = ['auc', 'accuracy', 'precision', 'recall', 'f1']
comparison = pd.DataFrame({
    'Original Features': [original_results[m] for m in metrics],
    'Engineered Features': [engineered_results[m] for m in metrics]
}, index=metrics)

# Calculate improvement
comparison['Improvement'] = comparison['Engineered Features'] - comparison['Original Features']
comparison['Improvement %'] = (comparison['Improvement'] / comparison['Original Features']) * 100

# Display comparison
display(comparison)

# Plot comparison
plt.figure(figsize=(12, 8))
comparison[['Original Features', 'Engineered Features']].plot(kind='bar')
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 6. Conclusions

### Key Improvements

1. **CustomerID Issue Fixed:**
   - Explicitly excluded CustomerID from feature engineering and model training
   - Prevented the model from learning patterns based on arbitrary identifiers

2. **Feature Selection Improvements:**
   - Implemented model-based feature selection for better capturing of non-linear relationships
   - Added mutual information option for alternative feature selection approach

3. **New Feature Categories:**
   - **Customer Value Features:** Capture lifetime value and revenue stability
   - **Trend Features:** Identify direction and acceleration of behavior changes
   - **Composite Risk Scores:** Aggregate multiple risk factors into meaningful scores
   - **Interaction Features:** Capture relationships between different risk factors

4. **Model Performance Balance:**
   - Implemented class weighting to balance precision and recall
   - Improved overall F1 score while maintaining good precision

### Business Implications

1. **Better Risk Identification:**
   - More accurate identification of high-risk customers
   - Earlier detection of churn signals through trend features
   - Prioritization of high-value customers at risk

2. **Actionable Insights:**
   - Specific risk factors identified for targeted interventions
   - Combination of risk factors (interactions) for more precise targeting
   - Customer value context for ROI-focused retention efforts

3. **Implementation Improvements:**
   - More robust feature engineering pipeline
   - Better handling of ID columns and categorical features
   - More flexible feature selection options

These improvements have significantly enhanced the churn prediction system's ability to identify at-risk customers while providing more context about the nature and severity of the risk.